In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
RU_train_path = "./Data/RU/train"
ES_train_path = "./Data/ES/train"

RU_dev_in_path = "./Data/RU/dev.in"
ES_dev_in_path = "./Data/ES/dev.in"

RU_dev_out1_path = "./Data/RU/dev.p1.out"
ES_dev_out1_path = "./Data/ES/dev.p1.out"

RU_dev_out2_path = "./Data/RU/dev.p2.out"
ES_dev_out2_path = "./Data/ES/dev.p2.out"

RU_dev_gold_path = "./Data/RU/dev.out"
ES_dev_gold_path = "./Data/ES/dev.out"

In [3]:
def load_data(filename):
    """Load data from file."""
    sentences = []
    sentence = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if line:  # line not empty
                tokens = line.split(' ')  # Split by space
                if len(tokens) >= 2:
                    tag = tokens[-1]  # Take the last token as tag
                    word = ' '.join(tokens[:-1])  # Combine all other tokens as word
                    sentence.append((word, tag))
                else:
                    print(f"Unexpected format on line {line_num}: {line}")
            else:  # empty line = end of a sentence
                if sentence:
                    sentences.append(sentence)
                    sentence = []
    return sentences


In [4]:
def estimate_emission_parameters(training_data):
    # Initialize counters
    emission_counts = {}  # Count(y -> x)
    tag_counts = {}       # Count(y)
    
    # Count occurrences
    for word, tag in training_data:
        # Update emission count
        if (tag, word) in emission_counts:
            emission_counts[(tag, word)] += 1
        else:
            emission_counts[(tag, word)] = 1
            
        # Update tag count
        if tag in tag_counts:
            tag_counts[tag] += 1
        else:
            tag_counts[tag] = 1
    
    # Calculate emission probabilities
    emission_probs = {}
    for (tag, word), count in emission_counts.items():
        emission_probs[(tag, word)] = count / tag_counts[tag]
    
    return emission_probs

In [5]:
def estimate_emission_parameters_with_unk(training_data, k=1):
    # Initialize counters
    emission_counts = {}  # Count(y -> x)
    tag_counts = {}       # Count(y)
    
    # Count occurrences
    for sentence in training_data:   
        for word, tag in sentence:   
            # Update emission count
            if (tag, word) in emission_counts:
                emission_counts[(tag, word)] += 1
            else:
                emission_counts[(tag, word)] = 1
                
            # Update tag count
            if tag in tag_counts:
                tag_counts[tag] += 1
            else:
                tag_counts[tag] = 1
                
    # Add #UNK# counts
    for tag in tag_counts:
        if (tag, "#UNK#") not in emission_counts:
            emission_counts[(tag, "#UNK#")] = 0
        emission_counts[(tag, "#UNK#")] += k
    
    # Calculate emission probabilities
    emission_probs = {}
    for (tag, word), count in emission_counts.items():
        if word == "#UNK#":
            emission_probs[(tag, word)] = k / (tag_counts[tag] + k)
        else:
            emission_probs[(tag, word)] = count / (tag_counts[tag] + k)
    
    return emission_probs


In [6]:
def predict_tag(word, emission_params):
    # Check if word was seen during training
    if word not in [w for tag, w in emission_params.keys()]:
        word = "#UNK#"
    
    # Predict tag based on maximum emission probability
    possible_tags = [tag for tag, w in emission_params.keys() if w == word]
    return max(possible_tags, key=lambda tag: emission_params[(tag, word)])


In [7]:
def predict_sentences(sentences, emission_params):
    predictions = []
    for sentence in sentences:
        sentence_predictions = []
        for word in sentence.split():  # Split sentence into words
            tag = predict_tag(word, emission_params)
            sentence_predictions.append((word, tag))
        predictions.append(sentence_predictions)
    return predictions


In [8]:
def write_predictions_to_file(predictions, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        for sentence in predictions:
            for word, tag in sentence:
                f.write(f"{word} {tag}")
            f.write("\n")  # separate sentences

In [9]:
def estimate_transition_parameters(training_data):
    transition_counts = {}  # Count(y_{i-1} -> y_i)
    tag_counts = {}  # Count(y_{i-1})
    
    for sentence in training_data:
        prev_tag = "START"
        for _, tag in sentence:
            if (prev_tag, tag) in transition_counts:
                transition_counts[(prev_tag, tag)] += 1
            else:
                transition_counts[(prev_tag, tag)] = 1
                
            if prev_tag in tag_counts:
                tag_counts[prev_tag] += 1
            else:
                tag_counts[prev_tag] = 1
            
            prev_tag = tag
        
        # Count for STOP after sentence ends
        if (prev_tag, "STOP") in transition_counts:
            transition_counts[(prev_tag, "STOP")] += 1
        else:
            transition_counts[(prev_tag, "STOP")] = 1
            
    transition_probs = {}
    for (tag1, tag2), count in transition_counts.items():
        transition_probs[(tag1, tag2)] = count / tag_counts[tag1]
    
    return transition_probs


In [34]:
def viterbi_algorithm(lines, emission_params, transition_params):
    n = len(lines)
    tags = list(set([tag for tag, word in emission_params.keys()] + ["START", "STOP"]))
    
    # Initialization
    viterbi = {}
    maxprob = {}
    for tag in tags:
        if tag == "START":
            viterbi[tag, 0] = 0  # log(1) = 0
        else:
            viterbi[tag, 0] = float('-inf')  # Represents log(0)
        maxprob[tag, 0] = None
    
    # Recursion
    for t in range(1, n + 1):
        word = lines[t-1]
        if word not in [w for t, w in emission_params.keys()]:
            word = "#UNK#"
        
        for tag in tags:
            max_log_prob = float('-inf')
            best_prev = None
            
            for prev_tag in tags:
                log_prob = viterbi[prev_tag, t-1]
                if (prev_tag, tag) in transition_params:
                    log_prob += math.log(transition_params[(prev_tag, tag)])
                else:
                    log_prob += float('-inf')  # Represents log(0)

                if (tag, word) in emission_params:
                    log_prob += math.log(emission_params[(tag, word)])
                else:
                    log_prob += float('-inf')  # Represents log(0)

                if log_prob > max_log_prob:
                    max_log_prob = log_prob
                    best_prev = prev_tag
            
            viterbi[tag, t] = max_log_prob
            maxprob[tag, t] = best_prev
    
    # Termination
    best_log_prob = float('-inf')
    best_prev = None
    for tag in tags:
        log_prob = viterbi[tag, n]
        if (tag, "STOP") in transition_params:
            log_prob += math.log(transition_params[(tag, "STOP")])
        else:
            log_prob += float('-inf')  # Represents log(0)
            
        if log_prob > best_log_prob:
            best_log_prob = log_prob
            best_prev = tag
            
    # Backtrack to get the best tag sequence
    tags = [best_prev]
    for t in range(n, 1, -1):
        best_prev = maxprob[best_prev, t]
        tags.append(best_prev)
    tags.reverse()
    
    return list(zip(lines, tags))

In [27]:
def predict_sentences_viterbi(sentences, emission_params, transition_params):
    predictions = []
    
    for sentence in sentences:
        # Split sentence into words
        words = sentence.split()
        sentence_predictions = viterbi_algorithm(words, emission_params, transition_params)
        predictions.append(sentence_predictions)
    
    return predictions


In [11]:
train_data_RU = load_data(RU_train_path)
train_data_ES = load_data(ES_train_path)

In [12]:
emission_params_RU = estimate_emission_parameters_with_unk(train_data_RU)
transition_params_RU = estimate_transition_parameters(train_data_RU)

emission_params_ES = estimate_emission_parameters_with_unk(train_data_ES)
transition_params_ES = estimate_transition_parameters(train_data_ES)

In [13]:
dev_data_RU = 0
dev_data_ES = 0
with open(RU_dev_in_path, 'r', encoding='utf-8') as file:
    dev_data_RU = [line.strip() for line in file]

with open(ES_dev_in_path, 'r', encoding='utf-8') as file:
    dev_data_ES = [line.strip() for line in file]

In [22]:
# Predict tags for dev data
predictions_RU = predict_sentences(dev_data_RU, emission_params_RU)
predictions_ES = predict_sentences(dev_data_ES, emission_params_ES)

In [15]:
# Write predictions to dev.p1.out
write_predictions_to_file(predictions_RU, RU_dev_out1_path)
write_predictions_to_file(predictions_ES, ES_dev_out1_path)

In [35]:
predictions_RU2 = predict_sentences_viterbi(dev_data_RU, emission_params_RU, transition_params_RU)
predictions_ES2 = predict_sentences_viterbi(dev_data_ES, emission_params_ES, transition_params_ES)


here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here

here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here

here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here

here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here

here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here

...

here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here

here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here

here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here

here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here

here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here?
here

In [33]:
# Write predictions to dev.p2.out
write_predictions_to_file(predictions_RU2, RU_dev_out2_path)
write_predictions_to_file(predictions_ES2, ES_dev_out2_path)

In [24]:
predictions_RU

[[('Интерьер', 'B-neutral')],
 [(',', 'O')],
 [('интерьер', 'B-positive')],
 [(',', 'O')],
 [('и', 'I-neutral')],
 [('еще', 'O')],
 [('раз', 'O')],
 [('интерьер', 'B-positive')],
 [('!', 'O')],
 [('!', 'O')],
 [('!', 'O')],
 [('общее', 'O')],
 [('цветовое', 'I-neutral')],
 [('решение', 'O')],
 [('и', 'I-neutral')],
 [('каждая', 'I-neutral')],
 [('деталь', 'I-neutral')],
 [('-', 'I-neutral')],
 [('просто', 'O')],
 [('восхитительны', 'I-neutral')],
 [('!', 'O')],
 [('!', 'O')],
 [('!', 'O')],
 [('особенно', 'O')],
 [('на', 'I-negative')],
 [('контрасте', 'I-neutral')],
 [('с', 'I-neutral')],
 [('популярным', 'I-neutral')],
 [('сейчас', 'O')],
 [('минимализмом', 'I-neutral')],
 [(',', 'O')],
 [('Дача', 'O')],
 [('-', 'I-neutral')],
 [('самое', 'O')],
 [('домашне-уютное', 'I-neutral')],
 [('и', 'I-neutral')],
 [('при', 'I-negative')],
 [('этом', 'O')],
 [('очень', 'O')],
 [('красивое', 'I-neutral')],
 [('место', 'B-positive')],
 [('.', 'O')],
 [],
 [('Средний', 'I-neutral')],
 [('счет', 'O

In [29]:
predictions_RU2

[[('Интерьер', 'O')],
 [(',', 'O')],
 [('интерьер', 'O')],
 [(',', 'O')],
 [('и', 'O')],
 [('еще', 'O')],
 [('раз', 'O')],
 [('интерьер', 'O')],
 [('!', 'O')],
 [('!', 'O')],
 [('!', 'O')],
 [('общее', 'O')],
 [('цветовое', 'O')],
 [('решение', 'O')],
 [('и', 'O')],
 [('каждая', 'O')],
 [('деталь', 'O')],
 [('-', 'O')],
 [('просто', 'O')],
 [('восхитительны', 'O')],
 [('!', 'O')],
 [('!', 'O')],
 [('!', 'O')],
 [('особенно', 'O')],
 [('на', 'O')],
 [('контрасте', 'O')],
 [('с', 'O')],
 [('популярным', 'O')],
 [('сейчас', 'O')],
 [('минимализмом', 'O')],
 [(',', 'O')],
 [('Дача', 'O')],
 [('-', 'O')],
 [('самое', 'O')],
 [('домашне-уютное', 'O')],
 [('и', 'O')],
 [('при', 'O')],
 [('этом', 'O')],
 [('очень', 'O')],
 [('красивое', 'O')],
 [('место', 'O')],
 [('.', 'O')],
 [],
 [('Средний', 'O')],
 [('счет', 'O')],
 [('оказался', 'O')],
 [('весьма', 'O')],
 [('приличным', 'O')],
 [('.', 'O')],
 [('но', 'O')],
 [('только', 'O')],
 [('сугубо', 'O')],
 [('из-за', 'O')],
 [('вина', 'O')],
 [(